## StyleGAN2

#### 라이브러리 다운로드

In [1]:
!pip install nnabla-ext-cuda120
!git clone https://github.com/sony/nnabla-examples.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.8
    Uninstalling Cython-3.0.8:
      Successfully uninstalled Cython-3.0.8


Cloning into 'nnabla-examples'...
remote: Enumerating objects: 9643, done.
remote: Counting objects: 100% (2201/2201), done.
remote: Compressing objects: 100% (880/880), done.
remote: Total 9643 (delta 1280), reused 2117 (delta 1261), pack-reused 7442
Receiving objects: 100% (9643/9643), 299.13 MiB | 35.99 MiB/s, done.
Resolving deltas: 100% (5182/5182), done.
Updating files: 100% (1711/1711), done.


###### 초기 설정 및 모듈 불러오기

In [1]:
# 런타임 다시 시작 후 아래 코드를 실행합니다.

%cd nnabla-examples/image-generation/stylegan2
!wget https://nnabla.org/pretrained-models/nnabla-examples/GANs/stylegan2/styleGAN2_G_params.h5

from generate import *  # ①
from IPython.display import Image, display

dl_context = get_extension_context("cudnn") # ②
nn.set_default_context(dl_context)  # ③
layer_count = 18    # ④
save_dir = 'results'    # ⑤

nn.load_parameters("styleGAN2_G_params.h5")

/content/nnabla-examples/image-generation/stylegan2
--2024-02-29 07:03:15--  https://nnabla.org/pretrained-models/nnabla-examples/GANs/stylegan2/styleGAN2_G_params.h5
Resolving nnabla.org (nnabla.org)... 108.156.107.116, 108.156.107.129, 108.156.107.87, ...
Connecting to nnabla.org (nnabla.org)|108.156.107.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121643776 (116M) [binary/octet-stream]
Saving to: ‘styleGAN2_G_params.h5’

styleGAN2_G_params. 100%[===================>] 116.01M  39.8MB/s    in 2.9s    

2024-02-29 07:03:19 (39.8 MB/s) - ‘styleGAN2_G_params.h5’ saved [121643776/121643776]



###### 이미지 생성을 위한 난수 선언

In [2]:
base_seed = 228
truncation_psi = 0.23
additive_seed = 195
image_num = 2

###### 난수를 바탕으로 단일 이미지 생성

In [3]:
random_gen = np.random.RandomState(base_seed)
latent_input = random_gen.randn(image_num, 512)

nn.set_auto_forward(True)

gen_noise = nn.NdArray.from_numpy_array(latent_input)
noise_list = [gen_noise for _ in range(2)]

gen_output = generate(image_num, noise_list, additive_seed, mix_after=7, truncation_psi=truncation_psi)

output_images = convert_images_to_uint8(gen_output, drange=[-1, 1])

for i in range(image_num):
    file_name = f'noise_seed{base_seed}_{i}.png'
    imsave(file_name, output_images[i], channel_first=True)
    display(Image(file_name, width=512, height=512))

Output hidden; open in https://colab.research.google.com to view.

###### 스타일 믹스를 위한 난수 선언

In [4]:
primary_seed = 600
secondary_seed = 500
mix_after = 7
auxiliary_seed = 500
truncation_psi = 0.5
rough_batch_size = 2
fine_batch_size = 4

###### 난수를 바탕으로 노이즈 벡터 생성

In [5]:
generator_1 = np.random.RandomState(primary_seed)
primary_noise = nn.NdArray.from_numpy_array(generator_1.randn(rough_batch_size, 512))

generator_2 = np.random.RandomState(secondary_seed)
secondary_noise = nn.NdArray.from_numpy_array(generator_2.randn(fine_batch_size, 512))

nn.set_auto_forward(True)

###### 노이즈 벡터를 바탕으로 여러 이미지를 생성하고, 스타일이 합성된 이미지들을 표 형태로 표현

In [6]:
mixed_images = []
for i in range(rough_batch_size):   # ①
    column_images = []
    for j in range(fine_batch_size):
        mixed_noises = [F.reshape(primary_noise[i], (1, 512)), F.reshape(secondary_noise[j], (1, 512))] # ②
        output_rgb = generate(1, mixed_noises, auxiliary_seed, mix_after, truncation_psi)   # ③
        column_images.append(convert_images_to_uint8(output_rgb, drange=[-1, 1])[0])    # ④
    column_images = np.concatenate([img for img in column_images], axis=2)  # ⑤
    mixed_images.append(column_images)
mixed_images = np.concatenate([img for img in mixed_images], axis=1)

noises_primary = [primary_noise, primary_noise] # ⑥
output_primary = generate(rough_batch_size, noises_primary, auxiliary_seed, mix_after, truncation_psi)  # ⑦
primary_image = convert_images_to_uint8(output_primary, drange=[-1, 1])
primary_image = np.concatenate([img for img in primary_image], axis=1)

noises_secondary = [secondary_noise, secondary_noise]   # ⑧
output_secondary = generate(fine_batch_size, noises_secondary, auxiliary_seed, mix_after, truncation_psi)
secondary_image = convert_images_to_uint8(output_secondary, drange=[-1, 1])
secondary_image = np.concatenate([img for img in secondary_image], axis=2)

blank_image = 255 * np.ones(output_secondary[0].shape).astype(np.uint8) # ⑨
combined_top_image = np.concatenate((blank_image, secondary_image), axis=2)
final_grid_image = np.concatenate((primary_image, mixed_images), axis=2)
final_grid_image = np.concatenate((combined_top_image, final_grid_image), axis=1)

imsave("stylegan2_grid.png", final_grid_image, channel_first=True)  # ⑩
display(Image("stylegan2_grid.png", width=256*(fine_batch_size + 1), height=256*(rough_batch_size + 1)))

Output hidden; open in https://colab.research.google.com to view.